# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [ ]:
# --- Imports and path setup ---
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

# Ensure local package is importable

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_with_fakes_like_input.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

## Configure the planner

The next cell defines a complete `PlannerConfig`. Every parameter of the
dataclass is included so the notebook doubles as documentation. Edit any value
to explore different strategies or hardware assumptions.

### Default Host Surface Brightness
- Exposure capping includes an optional default host SB fallback when per-target host inputs are missing.
- Enabled by default: `cfg.use_default_host_sb = True`.
- Default per-filter values: `cfg.default_host_mu_arcsec2_by_filter = {'u': 22.0, 'g': 22.0, 'r': 22.0, 'i': 22.0, 'z': 22.0, 'y': 22.0}`.
- To disable, set `cfg.use_default_host_sb = False`. Customize by editing `cfg.default_host_mu_arcsec2_by_filter`.


In [ ]:
# ---- Planner configuration (LSST Rubin parity) ----
filters = ["g", "r", "i", "z", "Y"]  # NOTE: capital 'Y' for LSST

from twilight_planner_pkg.config import PlannerConfig

cfg = PlannerConfig(
    filters=filters,
    exposure_by_filter={"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0, "Y": 5.0},

    # Optical + detector
    pixel_scale_arcsec=0.2,
    gain_e_per_adu=1.6,

    # Saturation policy (keep headroom)
    simlib_npe_pixel_saturate=80000,
    simlib_photflag_saturate=2048,
    use_default_host_sb=True,  # enable rest->obs host SB fallback

    # Discovery fallback (still on), but we prefer peak mags when z exists
    use_discovery_fallback=True,
    discovery_policy="atlas_priors",
    discovery_assumed_gr=0.0,
    discovery_margin_mag=0.2,

    # Peak-mag knobs
    H0_km_s_Mpc=70.0, Omega_m=0.3, Omega_L=0.7,
    MB_absolute=-19.36, SALT2_alpha=0.14, SALT2_beta=3.1,
    peak_extra_bright_margin_mag=0.3,
    Kcorr_approx_mag_by_filter={"g":0.0, "r":0.1, "i":0.2, "z":0.2, "Y":0.15},

    # Keep policy flexible in twilight
    allow_filter_changes_in_twilight=True,
)

# SIMLIB header consistency
cfg.simlib_filters = "ugrizY"      # keep Y uppercase
cfg.simlib_pixsize = 0.200
cfg.simlib_psf_unit = "PIXEL"      # SNANA expects PSF1/PSF2/PSFRATIO in PIXEL units

# Safety checks
assert abs(cfg.pixel_scale_arcsec - 0.2) < 1e-6
assert 1.4 <= cfg.gain_e_per_adu <= 1.8
assert cfg.simlib_npe_pixel_saturate == 80000


In [ ]:
# Use SNANA central-pixel fraction (Taylor expansion) for parity
from twilight_planner_pkg.photom_rubin import PhotomConfig
phot_defaults = PhotomConfig(
    pixel_scale_arcsec=cfg.pixel_scale_arcsec,
    gain_e_per_adu=cfg.gain_e_per_adu,
    npe_pixel_saturate=cfg.simlib_npe_pixel_saturate,
    npe_pixel_warn_nonlinear=int(0.8 * cfg.simlib_npe_pixel_saturate),
    use_snana_fA=True
)


In [ ]:
from functools import lru_cache
from twilight_planner_pkg import astro_utils as AU

@lru_cache(maxsize=4096)
def _peak_mag_cached(z: float, band: str) -> float:
    return AU.peak_mag_from_redshift(
        float(z), str(band).lower(),
        MB=cfg.MB_absolute, alpha=cfg.SALT2_alpha, beta=cfg.SALT2_beta,
        H0=cfg.H0_km_s_Mpc, Om=cfg.Omega_m, Ol=cfg.Omega_L,
        K_approx=cfg.Kcorr_approx_mag_by_filter.get(str(band).lower(), 0.0),
    )

# Route internal calls through the cache
AU.peak_mag_from_redshift = _peak_mag_cached


In [ ]:
# Diagnose catalog magnitude mapping for discovery fallback
from twilight_planner_pkg.examine import diagnose_mag_mapping
band_to_col, maglike_cols, preview = diagnose_mag_mapping(CSV_PATH, cfg)
print('Band→column mapping:', band_to_col)
print('Columns containing "mag":', maglike_cols[:10])
display(preview)



## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`CSV_PATH`, writes output CSVs to `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [ ]:
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

catalog_csv = "path/to/your_catalog.csv"
outdir = "out/twilight_runs"
start_date = "2024-01-01"
end_date   = "2024-01-03"

# Maintain legacy variable names for downstream cells
CSV_PATH = catalog_csv
OUTDIR = outdir
START_DATE = start_date
END_DATE = end_date

# Enable SIMLIB output (SNANA expects PSF_UNIT=PIXEL)
cfg.simlib_out = f"{outdir}/lsst_twilight.simlib"

pernight_df, nights_df = plan_twilight_range_with_caps(
    csv_path=catalog_csv,
    outdir=outdir,
    start_date=start_date,
    end_date=end_date,
    cfg=cfg,
    run_label="hybrid",
    verbose=True
)


In [ ]:
perSN_df

In [ ]:
# Show the nightly summary
nights_df


## Inspect outputs on disk


In [ ]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

In [ ]:
# Diagnose per-visit saturation contributions
from twilight_planner_pkg.examine import build_saturation_df
saturation_df = build_saturation_df(perSN_df, CSV_PATH, cfg)
# Save and show a compact view
sat_path = os.path.join(OUTDIR, f'saturation_{START_DATE}_to_{END_DATE}.csv')
saturation_df.to_csv(sat_path, index=False)
cols = ['SN','filter','t_exp_s','t_exp_s_base','src_mag','sky_mag_arcsec2','host_mu_arcsec2',
        'e_src','e_sky','e_host','total_e','sat_limit','sat_guard_applied','warn_nonlinear',
        'over_sat','over_sat_base']
display(saturation_df[cols].head(20))
print('Wrote:', sat_path)



In [ ]:
print("Filters used:", cfg.filters)
print("SIMLIB sat (NPE_PIXEL_SATURATE):", cfg.simlib_npe_pixel_saturate)
print("PSF_UNIT:", cfg.simlib_psf_unit)

# If SIMLIB was written, glance at header lines
if cfg.simlib_out:
    with open(cfg.simlib_out, "r") as fp:
        for i, line in enumerate(fp):
            if i > 25:
                break
            print(line.rstrip())
